In [1]:
import tensorflow as tf
import numpy as np
import pprint

In [2]:
batch_size = 1

num_classes = 3            #분류할 사전의 크기 

learning_rate = 0.01
sequence_length = 10         

output_dim = 3

In [3]:
X = tf.placeholder(tf.float32, [None, sequence_length,num_classes], name="X")
Y = tf.placeholder(tf.float32, [None, output_dim], name="Y")

keep_prob = tf.placeholder(tf.float32)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=num_classes, state_is_tuple=True)
cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
cell = tf.contrib.rnn.MultiRNNCell([cell]*2, state_is_tuple= True)

BatchSize = tf.placeholder(tf.int32, [], name='BatchSize')
initial_state = cell.zero_state(BatchSize, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X,initial_state=initial_state,dtype=tf.float32)

dense1 = tf.layers.dense(inputs=outputs[:,-1], units=sequence_length*output_dim, activation=tf.nn.relu)
dropout1 = tf.nn.dropout(dense1, keep_prob=keep_prob)

dense2 = tf.layers.dense(inputs=dropout1, units=sequence_length, activation=tf.nn.relu)
dropout2 = tf.nn.dropout(dense1, keep_prob=keep_prob)

dense3 = tf.layers.dense(inputs=dropout2, units=output_dim, activation=tf.nn.relu)
dropout3 = tf.nn.dropout(dense3, keep_prob=keep_prob)

Y_pred= tf.layers.dense(inputs=dropout2, units=output_dim)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y_pred, labels=Y))
lr = tf.placeholder(tf.float32,shape=(), name='learning_rate')
train = tf.train.AdamOptimizer(lr).minimize(cost)


In [4]:

dataX = [[[0, 2, 3],[11, 16, 1],[ 3, 4,1],[ 3, 4,1],[ 3, 4,1],
         [0, 2, 3],[11, 16, 1],[ 3, 4,1],[ 3, 4,1],[ 3, 4,1]]]

dataY = [[0, 0,1]]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l,results = sess.run([train, cost,Y_pred], feed_dict={X: dataX, Y:dataY, BatchSize: 1, keep_prob: 0.6, lr: 0.01})
    print(_, l)

None 1.140152
None 1.1389947
None 1.0808946
None 0.84831756
None 0.82459956
None 0.91267705
None 0.85309094
None 0.8375639
None 0.4728623
None 0.8380451
None 0.40819928
None 0.56393903
None 0.64694524
None 0.43906873
None 0.38298634
None 0.09569821
None 0.15190744
None 0.24158138
None 0.32292202
None 0.27297693
None 0.17815122
None 0.076063626
None 0.11809833
None 0.069929
None 0.30639854
None 0.11504755
None 0.37667274
None 0.16967522
None 0.012354041
None 0.00010442189
None 0.008077455
None 0.0074653225
None 1.442422e-05
None 0.002527854
None 0.006525399
None 0.005434027
None 0.00010716341
None 0.022832591
None 0.0007912366
None 0.22488669
None 0.0027103615
None 4.9589835e-05
None 0.027631631
None 0.11041233
None 0.0058160834
None 0.00063554116
None 0.07755629
None 0.0002362449
None 0.0047661746
None 0.022081932
None 0.012931417
None 0.0041743293
None 3.552374e-05
None 7.6293654e-06
None 5.5430784e-05
None 1.9788546e-05
None 0.29456443
None 0.008616649
None 0.00055512274
None 5.54307

In [ ]:
y_true = sess.run(tf.argmax(dataY,1))
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: dataX })

In [ ]:
y_pred

In [ ]:
results